# Informações importantes

Este notebook foi executado na plataforma Google Colaboratory, pois não consegui executar em minha máquina, exigia muito processamento.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from typing import List, Tuple
import numpy as np
import math

In [0]:
def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]

In [0]:
def generate_even_data(
    max_int: int, batch_size: int = 16
) -> Tuple[List[int], List[List[int]]]:
    
    max_length = int(math.log(max_int, 2))

    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    labels = [1] * batch_size

    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data

In [0]:
def convert_float_matrix_to_int_list(
    float_matrix: np.array, threshold: float = 0.5
) -> List[int]:
    return [
        int("".join([str(int(y)) for y in x]), 2) for x in float_matrix >= threshold
    ]

In [0]:
import torch.nn as nn

In [0]:
class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))

In [0]:
class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))

In [0]:
from typing import Tuple
import math

import torch
import torch.nn as nn


In [0]:
def train(
    max_int: int = 128,
    batch_size: int = 16,
    training_steps: int = 500,
    learning_rate: float = 0.001,
    print_output_every_n_steps: int = 10,
) -> Tuple[nn.Module]:
    
    input_length = int(math.log(max_int, 2))

    
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
    discriminator_optimizer = torch.optim.Adam(
        discriminator.parameters(), lr=learning_rate
    )

    
    loss = nn.BCELoss()

    for i in range(training_steps):
        generator_optimizer.zero_grad()

        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()

        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)

        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(
            generator_discriminator_out, torch.zeros(batch_size)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        if i % print_output_every_n_steps == 0:
            print(convert_float_matrix_to_int_list(generated_data))

    return generator, discriminator


In [12]:
train()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[38, 103, 38, 44, 39, 7, 38, 39, 70, 34, 39, 46, 100, 38, 76, 38]
[37, 3, 44, 7, 7, 44, 103, 3, 7, 44, 39, 74, 39, 7, 3, 6]
[108, 39, 42, 45, 12, 14, 108, 35, 14, 34, 7, 39, 39, 38, 10, 79]
[44, 39, 14, 108, 6, 99, 103, 39, 6, 15, 43, 44, 101, 36, 45, 38]
[39, 39, 74, 39, 3, 7, 14, 44, 47, 39, 7, 39, 108, 103, 108, 103]
[39, 46, 39, 45, 39, 105, 7, 103, 39, 72, 12, 103, 39, 36, 39, 39]
[43, 39, 43, 39, 111, 72, 103, 103, 2, 75, 40, 43, 109, 47, 71, 40]
[47, 109, 34, 47, 45, 43, 103, 107, 75, 11, 109, 43, 39, 35, 3, 109]
[109, 107, 45, 109, 105, 103, 38, 101, 44, 99, 40, 45, 101, 39, 39, 43]
[105, 109, 109, 41, 34, 39, 109, 35, 99, 103, 99, 40, 47, 35, 109, 41]
[41, 109, 43, 109, 107, 103, 103, 105, 105, 45, 47, 43, 109, 43, 105, 105]
[45, 105, 109, 103, 107, 39, 99, 39, 101, 99, 99, 41, 97, 43, 109, 109]
[105, 109, 105, 103, 105, 47, 109, 35, 103, 105, 103, 35, 40, 43, 109, 45]
[109, 41, 109, 105, 109, 105, 45, 105, 109, 43, 105, 103, 45, 40, 103, 35]
[43, 75, 109, 43, 103, 109, 43, 10

(Generator(
   (dense_layer): Linear(in_features=7, out_features=7, bias=True)
   (activation): Sigmoid()
 ), Discriminator(
   (dense): Linear(in_features=7, out_features=1, bias=True)
   (activation): Sigmoid()
 ))